In [1]:
# Importing useful libraries
import requests
from bs4 import BeautifulSoup
import time
import os
import asyncio
import aiohttp
import pandas as pd
import asyncio
from aiohttp import ClientSession, ClientResponseError

## 1.1. Get the list of master's degree courses

We start with the list of courses to include in your corpus of documents. In particular, we focus on web scrapping the MSc Degrees. Next, we want you to collect the URL associated with each site in the list from the previously collected list. The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in the first 400 pages (each page has 15 courses, so you will end up with 6000 unique master's degree URLs).

The output of this step is a .txt file whose single line corresponds to the master's URL.

---
Let's use BeautifulSoup and Requests to scrape the links related to universities present on the first 400 pages of the following website:
'https://www.findamasters.com/masters-degrees/msc-degrees'.

Note that it's possible to navigate to different pages by modifying the link in the final part, appending "/?PG=" + the page number.

We are saving the URLs in each line of txt file called 'course_links'. In this case there aren't exceptions to handle with.

In [ ]:
# let's create a file .txt called course_links which contains in every line the URL of the link

with open('course_links.txt', 'w') as file:
    # scraping first page
    response = requests.get('https://www.findamasters.com/masters-degrees/msc-degrees')
    soup = BeautifulSoup(response.text, 'html.parser')
    course_link = soup.find_all('a', class_='courseLink')
    for link in course_link:
        file.write(f"www.findamasters.com{link.get('href')}\n") # writing URLs in the txt file

    # if response.status_code == 200:
    for i in range(2, 401):
        # scraping pages from 2 to 400
        response = requests.get(f'https://www.findamasters.com/masters-degrees/msc-degrees/?PG={i}')
        soup = BeautifulSoup(response.text, 'html.parser')
        course_link = soup.find_all('a', class_='courseLink')
        # writing URL's in the txt file
        for link in course_link:
            file.write(f"www.findamasters.com{link.get('href')}\n")

        time.sleep(1)
        # adding a time.sleep of 1 second is important to avoid sending too many requests to the website

## 1.2. Crawl master's degree pages
Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its HTML in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the courses on page 1, page 2, ... of the list of master's programs.

**Tip**: Due to the large number of pages you should download, you can use some methods that can help you shorten the time. If you employed a particular process or approach, kindly describe it.



---
The proposed solution for this task involves creating 400 folders, with each folder dedicated to a web page that was scraped in the previous exercise. Inside each of these folders, the corresponding HTML contents of the 15 website will be stored.

The script is organized into three asynchronous functions:
- *get_info(url, session, folder, page_number)*, which performs an asynchronous HTTP GET request to a line (URL) of the txt file created in the exercise above. It returns the html page, written in a specific folder. It manages exceptions such as "Error 429: Too many requests to the website" and if it happens there's a time.sleep of 1 second, until a new get request is sent.
- *process_batch(urls_session, folder)*, which takes in input 15 urls and creates a list of asynchronous tasks, each corresponding to fetching HTML content from a URL in the given list using the *get_info* function, defined before. It uses asyncio.gather to concurrently execute all tasks and returns the results.
- *main(urls, batch_size, starting_folder)* which creates the path where all the html are put. It iterates through batches of URLs, creating a sub-folder for each batch and calling process_batch to asynchronously download and save HTML content for each URL in the batch.


The Python script is designed for asynchronous tasks using the aiohttp library to fetch HTML content from a list of URLs concurrently. 

In this particular case, working on multiple downloads at the same time are not so effective because the 6000 URLs are all from the same server, so we need to insert a time.sleep to handle "Too many requests". As a consequence, the code takes several hours to complete. To mitigate this, we introduced the *starting_folder* parameter in the main function. This allows us to resume the download process from where it left off, avoiding the need to recreate files and folders from scratch each time.

In [ ]:
async def get_info(url, session, folder, page_number):
    try:
        resp = await session.request(method="GET", url="https://" + url)
        resp.raise_for_status()
        html = await resp.text(encoding='utf-8')
        
        # Create the path in the corresponding folder
        file_path = os.path.join(folder, f"page_{int(page_number)}.html")
        
        # Write the html page in the right folder
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(html)
        
        return html
    
    except ClientResponseError as e:
        if e.status == 429: # Error 429: too many requests.
            print(f"Received 429 error. Too many requests. Waiting for a while...")
            await asyncio.sleep(1)  # Wait for 1 second before retrying
            return await get_info(url, session, folder, page_number)
        else:
            raise e  # Re-raise other ClientResponseError
        
async def process_batch(urls, session, folder):
    tasks = [get_info(url, session, folder, page_number = count) for count, url in enumerate(urls, start=1)]
    return await asyncio.gather(*tasks)

async def main(urls, batch_size=15, starting_folder=0):
    main_directory = "master_programs_html"
    os.makedirs(main_directory, exist_ok=True)
    
    async with ClientSession() as session:
        count_folder = starting_folder + 1
        for i in range(starting_folder*batch_size, len(urls), batch_size):
            # selecting the URLs from urls variable
            batch_urls = urls[i:i + batch_size]
            # Creating a sub-folder for every batch
            folder_name = os.path.join(main_directory, f"folder_{count_folder}")
            os.makedirs(folder_name, exist_ok=True)
            count_folder += 1
            # Downloading and writing file HTML in the batch
            await process_batch(batch_urls, session, folder_name)

if __name__ == "__main__":
    with open('course_links.txt', 'r') as file:
        urls = [line.strip() for line in file] # creating a list with the 6000 URLs from the lines of course_links.txt
        
    result = await main(urls, starting_folder = 360) # starting_folder represents the folder we are starting from

    for text in result:
        pass # text contains your html (text) response
    print("Download and organization of HTML pages completed.")


## 1.3 Parse downloaded pages
At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format is as follows:

- Course Name (to save as courseName): string;
- University (to save as universityName): string;
- Faculty (to save as facultyName): string
- Full or Part Time (to save as isItFullTime): string;
- Short Description (to save as description): string;
- Start Date (to save as startDate): string;
- Fees (to save as fees): string;
- Modality (to save as modality):string;
- Duration (to save as duration):string;
- City (to save as city): string;
- Country (to save as country): string;
- Presence or online modality (to save as administration): string;
- Link to the page (to save as url): string.

For each master's degree, you create a course_i.tsv file of this structure:

        courseName \t universityName \t  ... \t url
If an information is missing, you just leave it as an empty string.

---
First things first let's create the empty dataframe with the variables described above.

In [45]:
# Name dataframe columns

columns = [
    "courseName",
    "universityName",
    "facultyName",
    "isItFullTime",
    "description",
    "startDate",
    "fees",
    "modality",
    "duration",
    "city",
    "country",
    "administration",
    "url"
]

# Create a dataframe with the specific columns above
df = pd.DataFrame(columns=columns)

# Visualizza il DataFrame
df


,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url


Now we need to populate the dataframe, opening every html page from every folder created in the exercise 1.2.

After this operation we have to find specific elements in the html page, handling exceptions, for example if there's no matching element with the *find_all* function, variable will be "".

For every row of the dataframe we are also creating a .tsv file named as the master degree, containing the corresponding variables values.

In [ ]:
# Populate the dataframe
pth = "master_programs_html"
with open('course_links.txt', 'r') as file:
    urls = [line.strip() for line in file] # creating a list with the 6000 URLs from the lines of course_links.txt

if not os.path.exists("files_tsv"):
    os.mkdir("files_tsv")

fold = 0  # change this parameter if you need to start the population of the df not from folder 1, without losing progresses
count = (15 * fold) + 1
count_na_rows = 0
for folder in range(fold + 1, 401): # loop for every folder
    for file in range(1, 16): # loop for every file
        file_path = os.path.join(pth, f"folder_{folder}", f"page_{file}.html")
        with open(file_path, 'r', encoding='utf-8', errors='replace') as fl:
            soup = BeautifulSoup(fl, 'html.parser')
            if soup.title.text == r"FindAMasters | 500 Error : Internal Server Error":
                courseName = universityName = facultyName = isItFullTime = description = startDate = fees = modality = duration = city = country = administration = ""
                count_na_rows += 1
            else:
                courseName = soup.find("h1", {"class": "course-header__course-title"})
                if courseName is None:
                    courseName = ""
                else:
                    courseName = courseName.get_text(strip = True)
                universityName = soup.find("a", {"class": "course-header__institution"}).get_text(strip = True)
                facultyName = soup.find("a", {"class": "course-header__department"}).get_text(strip = True)
                extract = soup.find("span", {"class": "key-info__study-type"})
                if extract is None:
                    isItFullTime = ""
                else:
                    isItFullTime = extract.get_text(strip = True)
                description = soup.find("div", {"class": "course-sections__description"}).find("div", {"class": "course-sections__content"}).get_text(strip = True)
                startDate = soup.find("span", {"class": "key-info__start-date"}).get_text(strip = True)
                # some entries do not have this field
                extract = soup.find("div", {"class": "course-sections__fees"})
                if extract is None:
                    fees = ""
                else:
                    fees = extract.find("div", {"class": "course-sections__content"}).get_text(strip = True)
                modality = soup.find("span", {"class": "key-info__qualification"}).get_text(strip = True)
                duration = soup.find("span", {"class": "key-info__duration"}).get_text(strip = True)
                city = soup.find("a", {"class": "course-data__city"}).get_text(strip = True)
                country = soup.find("a", {"class": "course-data__country"}).get_text(strip = True)
                extract1 = soup.find("a", {"class": "course-data__online"})
                extract2 = soup.find("a", {"class": "course-data__on-campus"})
                if extract1 is None and extract2 is None:
                    administration = ""
                elif extract2 is None:
                    administration = extract1.get_text(strip = True)
                elif extract1 is None:
                    administration = extract2.get_text(strip = True)
                else:
                    administration = extract1.get_text(strip = True) + " & " + extract2.get_text(strip = True)
                    
            url = urls[count-1]
            new_row_data = {"courseName": courseName,
                            "universityName": universityName,
                            "facultyName": facultyName,
                            "isItFullTime": isItFullTime,
                            "description": description,
                            "startDate": startDate,
                            "fees": fees,
                            "modality": modality,
                            "duration": duration,
                            "city": city,
                            "country": country,
                            "administration": administration,
                            "url" : url
                            }
            df = pd.concat([df, pd.DataFrame([new_row_data])], ignore_index=True)
            file_path = os.path.join('files_tsv', f'course{count}.tsv')
            pd.DataFrame([new_row_data]).to_csv(file_path, sep='\t', index=False)
            print(f"Created course{count}.tsv, {count}/6000, folder = {folder}, file = {file}")
            count += 1

print(f"There are {count_na_rows} links corrupted.")

Now let's check if we have considered all the 6000 rows.

In [47]:
df.shape[0]

6000

And print the first and the last 10 rows of the dataframe created.

In [49]:
df.head(10)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/3d...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total)International: £34,750 (Total)",MSc,1 year full time,Leeds,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ac...
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,Develop in-depth knowledge of accounting and f...,September,Please see the university website for further ...,MSc,1 year full-time,Bath,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ac...
3,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ac...
4,"Accounting, Financial Management and Digital B...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career wit...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ac...
5,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time&Part time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ad...
6,Advanced Chemical Engineering - MSc,University of Leeds,School of Chemical and Process Engineering,Full time,The Advanced Chemical Engineering MSc at Leeds...,September,"UK: £13,750 (Total)International: £31,000 (Total)",MSc,1 year full time,Leeds,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ad...
7,Advanced Physiotherapy Practice - MSc,Glasgow Caledonian University,School of Health and Life Sciences,Full time&Part time,Progress your career as a physiotherapist with...,"January, September",Please see the university website for further ...,MSc,1 Year Full Time / 2-3 Years Part Time,Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ad...
8,Agricultural Sciences - MSc (Agriculture and F...,University of Helsinki,International Masters Degree Programmes,Full time,Goal of the pro­grammeWould you like to be inv...,September,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,On Campus,www.findamasters.com/masters-degrees/course/ag...
9,"Agricultural, Environmental and Resource Econo...",University of Helsinki,International Masters Degree Programmes,Full time,Goal of the pro­grammeAre you looking forward ...,September,Tuition fee per year (non-EU/EEA students): 15...,MSc,2 years,Helsinki,Finland,On Campus,www.findamasters.com/masters-degrees/course/ag...


In [50]:
df.tail(10)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
5990,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programmeis des...,"September, January","€29,900/year","MBA,MSc",1 Year,Barcelona,Spain,On Campus,www.findamasters.com/masters-degrees/course/ma...
5991,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Full time,Front-end DevelopmentatHarbour.Space Universit...,"September, January","€29,900/year",MSc,1 year,Barcelona,Spain,On Campus,www.findamasters.com/masters-degrees/course/ma...
5992,Masters of Science in Business,Oregon State University,School of Business,Full time&Part time,Our Master of Science in Business (MSB) will g...,See Course,You can find more information here:Please see ...,"MSc,MBA",12 months,Corvallis,USA,Online,www.findamasters.com/masters-degrees/course/ma...
5993,"Masters of Science in Business, Supply Chain A...",Oregon State University,School of Business,Full time&Part time,Master of Science in Business (MSB)Our Master ...,See Course,You can find more information here:Please see ...,MSc,12 months,Corvallis,USA,Online,www.findamasters.com/masters-degrees/course/ma...
5994,"Masters Program in Climate Change, Agriculture...",University of Galway,Ryan Institute,Full time,The world’s climate is rapidly changing due to...,September,Please seePlease see the university website fo...,MSc,1 year,Galway,Ireland,On Campus,www.findamasters.com/masters-degrees/course/ma...
5995,Masters's in Digital Politics and Governance,European School of Political and Social Scienc...,Masters Programs,Full time,Digitalisation is a critical issue in today’s ...,See Course,Please see the university website for further ...,MSc,2 years,Lille,France,On Campus,www.findamasters.com/masters-degrees/course/ma...
5996,Material Culture & Artefact Studies - MSc/PgDip,University of Glasgow,College of Arts & Humanities,Full time&Part time,Material culture and artefact studies combines...,September,Please see the university website for further ...,"MSc,PGDip","9-12 months full-time, 18-24 months part-time",Glasgow,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ma...
5997,Material Culture and Gàidhealtachd History MSc,University of the Highlands and Islands,"Arts, Humanities and Business",Part time,"This ground-breaking, internationally acclaime...",September,Please see the university website for further ...,"PGDip,PGCert,MSc",3-6 Years Part-time,Inverness,United Kingdom,Online,www.findamasters.com/masters-degrees/course/ma...
5998,Materials and Manufacturing,Jonkoping University,Masters Programmes,Full time,Numerical methods and knowledge about the rela...,See Course,Please see the university website for further ...,MSc,2 Years Full Time,Jonkoping,Sweden,On Campus,www.findamasters.com/masters-degrees/course/ma...
5999,Materials and Molecular Modelling MSc,University College London,Department of Chemistry,Full time,Register your interest in graduate study at UC...,September,"Full time - £14,100",MSc,1 year full time,London,United Kingdom,On Campus,www.findamasters.com/masters-degrees/course/ma...
